<center> Universidade Federal de Minas Gerais </center>
<center> Departamento de Ciencia da Computação </center>
<center> DCC029/868 – Processamento de Imagens Digitais </center>
<center> Prof. Jefersson Alex dos Santos (jefersson@dcc.ufmg.br) </center>
<center> Monitor Caio Cesar Viana da Silva (caiosilva@ufmg.br) </center>

<h1 align="center">TP2: Classificação de Imagens com Banco de Filtros</h1>

<h4 align="center">Alison de Oliveira Souza - 2012049316</h4>
<h4 align="center">Yuri Diego Santos Niitsuma - 2011039023</h4>

## <center>Documentação</center>

#### <center>Introdução:</center>
Neste trabalho, abordamos a classificação de imagens de um subconjunto do dataset MNIST - que contém apenas os números 3 e 6 - à partir da definição e da aplicação de um conjunto de filtros de imagens. Esses filtros são construídos utilizando uma rede neural, que simulará filtros aleatórios, inicialmente aplicados em uma parte do subconjunto citado - no nosso caso será de 80% do conjunto inicial, que chamamos de subconjunto de treino - para que sejam obtidas características que nos possibilitem atribuir as imagens a alguma das classes obtidas durante o treino. Após isso, utilizamos a outra parte do subconjunto inicial - os 20% restante que chamamos de subconjunto de teste - para validar nosso banco de filtros de extração de características, utilizando um classificador para tentar identificar a qual classe pertence uma imagem do subconjunto de teste, ou seja, qual número está representado nesta imagem.

A imagem abaixo mostra um exemplo do funcionamento de nossa rede neural utilizada para o treinamento.

![](image.png)

#### <center>Metodologia:</center>
Para facilitar o entendimento do nosso projeto, dividimos nosso trabalho em partes. Ao longo desse notebook iremos adicionar células markdown com pedaços da nossa documentação entre cada parte do código, facilitando a leitura e a atribuição de cada etapa da documentação a seu respectivo código.


###### Import das bibliotecas utilizadas e definição de funções auxiliares

In [ ]:
import numpy as np
import os
import random
from skimage import io, img_as_float
from scipy import ndimage
import matplotlib.pyplot as plt

%matplotlib inline  

###### Função auxiliar para exibir as imagens

In [ ]:
def show(img, cmap=None):
    cmap = cmap or plt.cm.gray
    fig, ax = plt.subplots(1, 1, figsize=(8, 6))
    ax.imshow(img, cmap=cmap)
    ax.set_axis_off()
    plt.show()

###### Etapa 1: Funções de leitura do conjunto de dados fornecido
As descrições de cada função se encontra como comentários no início de cada uma delas.

In [ ]:
DATASET_FOLDER = './MNIST/'
SHAPE = (28, 28)

def get_files_paths(number):
    '''
    Retorna todos os arquivos de uma determinada pasta
    MNIST/<number>.
    '''
    files = os.listdir(DATASET_FOLDER + str(number))
    for i in range(len(files)):
        files[i] = DATASET_FOLDER + str(number) + '/' + files[i]
    return files


def open_image(filepath=None, as_gray=True):
    """
    Abre a imagem e retorna em formato float.
    """
    if filepath is None:
        raise Exception('No filepath passed in open_image!')
    return img_as_float(io.imread(filepath, as_gray=as_gray))


def load_data(numbers=[]):
    """
    Constrói as listas de imagens e de informações sobre cada
    imagem.
    data --> lista de imagens
    data_id --> lista de classes das imagens (target) 
    val_id --> lista de posições de cada imagem
    """
    size = len(numbers)
    data = list()
    data_id = list()
    val_id = list()

    count = 0

    for number in numbers:
        filename_list = get_files_paths(number)
        for filename in filename_list:
            data.append(open_image(filename))
            an_array = np.zeros((size, 1), dtype=float)
            val_id.append(count)
            an_array[count] = 1.0
            data_id.append(an_array)
        
        count += 1

    return (data, data_id, val_id)


def load_data_wrapper(numbers=[]):
    """
    Gera as listas criadas por load_data() e faz uma formatação
    simples dos dados.
    """
    samples_inputs, samples_results, samples_validation = load_data(numbers=numbers)
    samples_inputs = [np.reshape(x, (784, 1)) for x in samples_inputs]    
    samples_data = list(zip(samples_inputs, samples_results, samples_validation))
    return samples_data


###### Funções auxiliares.
Aqui são declaradas funções extras que são utilizadas na classe Network.

In [ ]:
def sigmoid(z):
    """
    Função sigmoid. É utilizada como normalizador para o
    perceptron. Para saber mais acesse:
    https://pt.wikipedia.org/wiki/Fun%C3%A7%C3%A3o_sigm%C3%B3ide
    """
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

###### Definição da classe Network
Nesta classe, definimos a rede convolucional utilizada pelo nosso projeto. Para isso, utilizamos um algoritmo de aprendizado com gradiente descendente estocástico em uma rede neural. Os gradientes são calculados utilizando backpropagation.

In [ ]:
class Network(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def show_heights(self):
        """
        Exibe a combinação dos filtros utilizados.
        Inicialmente, começa com pesos aleatórios.
        """
        for layer in self.weights:
            # print('Shape:' + str(layer.shape))
            show(layer)
            # for each_neuron in layer:
            #     show(each_neuron)

    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, data, epochs, mini_batch_size, eta):
        """
        Método que utiliza o gradiente descendente estocástico
        para fazer o treinamento da rede neural e classificação das
        imagens. Os gradientes são calculados e os pesos das conexões
        são atualizadas utilizando backpropagation. 
        Neste método é feito a separação dos conjuntos de treino e de
        testes, o treinamento de acordo com o conjunto de treino
        escolhido e a classificação do conjunto de testes.
        """
        print('Total of ' + str(len(data)) + ' images for dataset!')
        random.shuffle(data)
        # Pega 80% do dataset pra utilizar em treinamento, o restante é
        # utilizado nos testes.
        n = int(0.8 * len(data))
        training_data, test_data = data[:n], data[n:]

        n_test = len(test_data)
        for j in range(epochs):
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                # print('Efetuando treinamento!')
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                evaluate = self.evaluate(test_data)
                print("Iteração " + str(j) + ": " +
                      str(self.evaluate(test_data)) + " / " + str(n_test))
                print('\tAcurácia: ' + str(format(evaluate/n_test * 100, '.2f')) + '%')
            else:
                print("Epoch " + str(j) + " complete")
            # Atualiza as listas de treino e teste.
            random.shuffle(data)
            n = int(0.8 * len(data))
            training_data, test_data = data[:n], data[n:]

    def update_mini_batch(self, mini_batch, eta):
        """
        Atualiza os pesos e biases da rede aplicando o gradiente
        descendente estocástico usando backpropagation.
        """
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y, z in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """
        Retorna uma tupla que representa o gradiente para
        a função de custo.
        """
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x]  # list to store all the activations, layer by layer
        zs = []  # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """
        Retorna o número de entradas de teste para as quais
        a rede neural produz o resultado correto.
        """
        test_results = [(np.argmax(self.feedforward(x)), z)
                        for (x, y, z) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        return (output_activations-y)

###### Aplicação da rede neural.
Neste bloco, estamos aplicando as funções definidas acima para o treinamento.

Inicialmente, delimitamos os conjuntos de dados que serão lidos - no nosso caso os números 3 e 6 - e carregamos as imagens correspondentes.

Após isso, definimos a quantidade de neurônios que usaremos por camada. Decidimos por 28x28 neurônios em duas camadas, sendo fully-connected - ou seja, todos se conectam - para que cada um dos neurônios sirva de filtro para cada um dos pixels das imagens de entrada.

Depois, inicializamos a rede convolucional e chamamos a função SGD que começa realizando o treinamento da rede neural e depois faz a classificação do conjunto de testes. Nesta função, é efetuada as iterações, passando como parâmetro os dados com os labels do dataset, a quantidade de iterações, a quantidade parcial para utilizar no treinamento, e o parâmetro de peso na aprendizagem - ou seja - a magnitude do deslocamento no domínio dado pelo gradiente.

A cada iteração, essa função escolhe aleatoriamente 80% dos dados de entrada para serem os dados de treino e 20% para serem os dados de testes.

Ao final de cada iteração, a função irá imprimir a acurácia de cada iteração e irá ajustar a rede neural para que a próxima iteração seja otimizada através do aprendizado obtido. Dessa forma, a tendência é que a cada iteração a acurácia melhore. Porém, algumas variações podem ocorrer devido a escolha aleatória dos conjuntos de treino e teste.

In [ ]:
# Determina quais são os conjuntos que serão lidos
NUMBER_SETS=[3,6]

# Carrega as imagens de entrada.
data = load_data_wrapper(numbers=NUMBER_SETS)

# Determina quantos neurônios estará no layer
layers = [28, 28]

# Inicializa a rede convulocional
net = Network([784, *layers, len(NUMBER_SETS)])
net.show_heights()

# Efetua as iterações de treinamento e classificação (15 neste caso)
net.SGD(data, 15, 10, 3.0)